In [ ]:
#| default_exp shallow

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from fastcore.test import *
import numpy as np

# Shallow water corrections (shallow)

> Addressing additional resistance caused by changes to viscosity and additional sinkage in shallow water

When ships pass through shallow water, they experience higher resistance due to changes in the viscous friction and the impact of squatting or sinkage in the water. These effects are due to the water passing under the ship being constricted and thus having a higher velocity. The change in velocity results in a difference in the buoyancy of the ship in accordance with the Bernoulli principle, which states that 

$$P +\frac{1}{2} \rho v^2 + \rho g h = \textrm{Constant}$$

Where $P$ is pressure, $\rho$ is water density, $v$ is the speed through water,  $g$ is gravity, and $h$ is the distance from the reference plane. As can be seen, an increase in water velocity must be balanced by a change in at least one of the other two variables. Changes in pressure result in the ship's buoyancy being reduced, causing it to sink slightly or squat. As the coefficient of friction experienced by the vessel is proportional to the square of the speed through water, the frictional resistance on the ship increases. A secondary effect is that the sinkage exposes an additional surface area of the ship to the water, increasing the friction effects further. Finally, as the ship's hull gets closer to the sea floor, the boundary layer between the hull and the free-flowing water thickens and becomes more turbulent increasing resistance. As a result of the additional resistance, the engine is required to deliver more power to maintain speed.

As such, when calculating the performance of a ship in shallow water, care needs to be taken to adjust for the effects this produces or risk misinterpreting the performance results.

## Importing the module

The module is called `shallow` import as

`from pyseatrials.shallow import *`

## Raven shallow water correction

Although there are several correction approaches to correcting for the effect of shallow water this library uses the method introduced by Raven in 2016 and described fully in [Raven 2019](https://doi.org/10.1016/j.oceaneng.2019.106343). The correction has two component resistance due to changes in viscous friction, resistance due to sinkage.

Consider the following

$R_{V, \,\textrm{deep}} = C_v'\frac{1}{2} \rho  V_S^2 S$

Is the viscous friction in deep water, where $C_v'$ is the coeficient of viscous friction, $\rho$ is the density of water, $V_S$ is the speed through water, and $S$ is the wetted surface area

The correction due to the changes in viscous resistance are given as 

$$ R_{V} = R_{V, \,\textrm{deep}} 0.57 (\frac{T_M}{h})^{1.79},$$

Where, $T_M$ is the draught at midship and $h$ is the water depth. 

The sinkage is calculated using the following method,

$$d(\textrm{sinkage}) = 1.46 \frac{\nabla}{L_\textrm{pp}^2}\left[ 
\frac{\textrm{Fr}_\textrm{h}^2}{\sqrt{1 - \textrm{Fr}_\textrm{h}^2}} -
 \frac{\textrm{Fr}_\textrm{hd}^2}{\sqrt{1 - \textrm{Fr}_\textrm{hd}^2}}
\right],$$

where L_\textrm{pp} is the length between perpendiculars, $\nabla$ is $\nabla = L_\textrm{pp} B T_M C_B$, and $B$ is the ships beam (m), $C_B$ is the dimensionless block coefficient, $\textrm{Fr}_\textrm{hd}$ is the Froude number where $h = 0.3 L_\textrm{pp}$, $\textrm{Fr}_\textrm{h}$ is the Froude number where $h$ is water depth. It should be noted that $d(\textrm{sinkage}) \geq 0 $. 

Additional displacement due to sinkage is then found such that

$$\delta \nabla =  \begin{cases}
d(\textrm{sinkage})\frac{A_W}{\nabla} & \text{if } d(\textrm{sinkage})\frac{A_W}{\nabla} < 0.05 \\
0.05 & \text{if } d(\textrm{sinkage})\frac{A_W}{\nabla} \geq 0.05
\end{cases} $$

Where $A_W$ is the water plane area. With $\delta nabla$ now known we find the sinkage value using

$$\textrm{rsink} = (1- \delta \nabla)^\frac{2}{3} $$

We then correct the power shallow water power to deep water power using 

$$P_\textrm{D,deep} = \frac{P_\textrm{D, shallow}}{\textrm{rsink}} - \frac{\Delta R_V V_S}{\eta_{Did}}$$

Finally we perform a check to ensure that

$$R_{V, \,\textrm{deep}}  \leq  \frac{P_\textrm{D,deep} \eta_\textrm{Did}}{V_s}$$

if the above condition is not satisfied $R_{V, \,\textrm{deep}}$ is set to the upper limit and the process is repeated.




In [ ]:
#| export
import numpy as np

def shallow_water_correction(coef_friction: float, #the coefficient of viscous friction [none]
                             water_density: float, # Water density [kg/m^3]
                             stw: float,  # speed through water [m/s^2]
                             wetted_surface_area: float, # The wetted surface area of the ships hull [m^2]
                             draught: float, #The draught at mid-ship [m]
                             water_depth: float, # The depth of the water [m]
                             L_pp: float, #The length between perpendiculars of the ship [m]
                             beam: float, #The beam of the ship [m]
                             C_B: float, #The block coefficient of the ship [none]
                             power: float, #The engine power [kW]
                             propulsive_efficiency: float,  #The propulsive efficiency of the propeller [none]
                             R_V_deep=None #The viscous friction experienced by the ship, this is left as none and used internally by the function
                             ) -> tuple[float, float, float]: # Returns 3 values the equivalent deep water power, the sinkage, the viscous resistance correction
    """
    Perform Raven corrections for shallow water performance
    """
    # Calculate viscous friction in deep water if not provided
    if R_V_deep is None:
        R_V_deep = coef_friction * 0.5 * water_density * stw**2 * wetted_surface_area

    # Calculate the viscous resistance correction
    R_V = R_V_deep * 0.57 * (draught /water_depth)**1.79

    # Calculate the sinkage
    displacement = L_pp * beam * draught * C_B
    Fr_hd = stw / np.sqrt(9.81 * 0.3 * L_pp)
    Fr_h = stw / np.sqrt(9.81 *water_depth)
    sinkage = 1.46 * displacement / L_pp**2 * ((Fr_h**2 / np.sqrt(1 - Fr_h**2)) - (Fr_hd**2 / np.sqrt(1 - Fr_hd**2)))
    #sinkage = np.maximum(sinkage, 0)

    # Calculate additional displacement due to sinkage
    A_W = L_pp * beam
    delta_displacement = np.minimum(sinkage * A_W / displacement, 0.05)

    # Calculate rsink
    rsink = (1 - delta_displacement)**(2/3)

    # Calculate deep water power
    P_D_deep = (power / rsink) - (R_V * stw / propulsive_efficiency)

    # Check if the condition is met
    condition_met = R_V_deep <= P_D_deep * propulsive_efficiency / stw

    # If the condition is not met, set R_V_deep to the upper limit and repeat the process
    if not np.all(condition_met):
        R_V_deep = np.where(condition_met, R_V_deep, P_D_deep * propulsive_efficiency / stw)
        #The recursion only happens once as P_D_deep * propulsive_efficiency / stw is constant
        P_D_deep, sinkage, R_V = shallow_water_correction(coef_friction, water_density, stw, wetted_surface_area, draught,water_depth, L_pp, beam, C_B, power, propulsive_efficiency, R_V_deep)

    return P_D_deep, sinkage, R_V


The example below shows a simple use case

In [ ]:
coef_friction = 0.075
water_density = 1025.0
stw = np.array([8.0, 10.0, 12.0])  # m/s
wetted_surface_area = 1800.0  # m^2
draught = 10.0  # m
water_depth = np.array([15.0, 20.0, 25.0])  # m
L_pp = 120.0  # m
beam = 20.0  # m
C_B = 0.7
power = np.array([6000.0, 8000.0, 10000.0])  # kW
propulsive_efficiency = 0.6

# Call the function with the example inputs
P_D_deep, sinkage, R_V = shallow_water_correction(coef_friction, water_density, stw, wetted_surface_area, draught, water_depth, L_pp, beam, C_B, power, propulsive_efficiency)

# Print the results
print("Equivalent deep water power:", (P_D_deep).astype(int), "kW")
print("Sinkage:", np.round(sinkage,2), "m")
print("Viscous resistance correction:", (R_V).astype(int), "N")

Equivalent deep water power: [4497009 3139757 2444459] kW
Sinkage: [0.64 0.67 0.65] m
Viscous resistance correction: [-336810 -187888 -121705] N


In [ ]:
#| hide
test_case1 = {
    "coef_friction": 0.075,
    "water_density": 1025.0,
    "stw": 6.0,
    "wetted_surface_area": 1800.0,
    "draught": 10.0,
    "water_depth": 20.0,
    "L_pp": 120.0,
    "beam": 20.0,
    "C_B": 0.7,
    "power": 8000.0,
    "propulsive_efficiency": 0.6
}

# Call the function with the example inputs
P_D_deep1, sinkage1, R_V = shallow_water_correction(**test_case1 )

# Print the results
test_close(P_D_deep1.astype(int), 683481, eps=1)
test_close(sinkage1, 0.163, eps=1e-3)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()